# 02. Tratamendo dos dados

Neste _notebook_ vou realizar a união dos dois datasets que eu extraí.

## Inicialização

Carregando as bibliotecas e as bases de dados que serão utilizadas nesse notebook.

In [1]:
import locale

import numpy as np
import pandas as pd

from operator import index
from datetime import datetime

In [2]:
# dataframe de indices socioeconomicos

df_se = pd.read_excel("./HDR21-22_Statistical_Annex_HDI_Table.xlsx")
df_feriados = pd.read_csv("./feriados.csv")

In [3]:
df_feriados.head(10)

,país,data,nome,tipo
0,afghanistan,15 de Fev,Liberation Day,Public Holiday
1,afghanistan,20 de Mar,March Equinox,Season
2,afghanistan,21 de Mar,Nauruz,Observance
3,afghanistan,2 de Abr,First Day of Ramadan,Public Holiday
4,afghanistan,28 de Abr,Afghan Victory Day,Public Holiday
5,afghanistan,1 de Mai,Eid al-Fitr,Public Holiday
6,afghanistan,1 de Mai,Labor Day,Public Holiday
7,afghanistan,2 de Mai,Eid al-Fitr Holiday,Public Holiday
8,afghanistan,3 de Mai,Eid al-Fitr Holiday,Public Holiday
9,afghanistan,21 de Jun,June Solstice,Season


dentre as variáveis presentes no dataframe relacionado aos feriados dos países temos:

- **país**: País onde aquele feriado ocorre;

- **data**: Essa variável pode ser um pouco enganosa, pois em alguns feriados ocorrem de acordo com um dia específico da semana combinado com algumas condições, como por exemplo a Páscoa Cristã, que é celebrada no primeiro domingo após a primeira lua cheia que ocorre depois do equinócio de primavera/outono ([fonte: Mundo Educação](https://mundoeducacao.uol.com.br/pascoa/data-pascoa.htm));

- **nome**: Nome do feriado que é celebrado no dia;

- **tipo**: Os tipos de feriado presentes no dataframe são diversos, como feriados relacionados a estação (Solstícios, Equinócios), feriados nacionais (aniversário do imperador do Japão, dias da independência ou datas comemorativas específicas), feriados apenas para trabalhadores do setor público (dia do setor público em Costa Rica), feriados estaduais... 

In [4]:
df_se.head(10)

,HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank.1
0,1,Switzerland,0.962,83.9872,16.500299,13.859660,66933.00454,5,3
1,2,Norway,0.961,83.2339,18.185200,13.003630,64660.10622,6,1
2,3,Iceland,0.959,82.6782,19.163059,13.767170,55782.04981,11,2
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.226210,62606.84540,6,4
4,5,Australia,0.951,84.5265,21.054590,12.726820,49238.43335,18,5
5,6,Denmark,0.948,81.3753,18.714800,12.960490,60364.78595,6,5
6,7,Sweden,0.947,82.9833,19.418530,12.609720,54489.37401,9,9
7,8,Ireland,0.945,81.9976,18.945221,11.582223,76168.98443,-3,8
8,9,Germany,0.942,80.6301,17.010139,14.090967,54534.21682,6,7
9,10,Netherlands,0.941,81.6873,18.693165,12.581630,55979.41100,3,10


No banco de dados acima, temos as seguintes variáveis:

- **HDI rank** : Ranking atual de acordo com o IDH do País;

- **Country** : País referente aos índices indicados;

- **Human Development Index (HDI)** : valor do IDH recente (2020 - 2022)

    (para saber mais sobre o cálculo do IDH [clique aqui](https://educacao.uol.com.br/disciplinas/geografia/indice-de-desenvolvimento-humano-idh-mede-nivel-de-qualidade-de-vida.htm#:~:text=Como%20funciona%20o%20IDH,o%20desenvolvimento%20é%20considerado%20alto.))

- **Life expectancy at birth** : Expectativa de vida;

- **Expected years of schooling** : número de anos de escolaridade que uma criança/adolescencia passa para concluir toda sua educação;

- **Mean years of schooling** : Número médio de anos que um cidadão estuda;

- **Gross national income (GNI) per capita** : ????

As últimas duas colunas serão excluídas por não serem de interesse a este estudo.


## Tratamento dos dados

Vamos dividir essa etapa em 3 partes:
- tratamento do primeiro dataframe;
- tratamento do segundo dataframe;
- Agregação dos dataframes;

Iniciando pelo que aparenta ser o mais simples, o tratamento dos dados referentes aos feriados de cada país. Como o objetivo do projeto é analisar o impacto de feriados que realmente [MODIFICAR] TIRAM DIAS LETIVOS/DÃO DIAS DE FOLGA e os índices socioêconomicos, vamos realizar um agrupamento pelo tipo de feriado, deixando assim o trabalho de ciência de dados mais simples resumindo nossos dados

In [13]:
# Analizando os tipos de feriado
df_feriados['tipo'].value_counts()

Public Holiday                1964
Observance                    1208
National holiday              1011
Season                         920
United Nations observance      177
                              ... 
Orthodox, Optional holiday       1
Working day (replacement)        1
Academic Holiday                 1
Public Sector                    1
Prescribed Day of Rest           1
Name: tipo, Length: 123, dtype: int64

Existem 123 tipos diferentes de feriado, vamos analisar os mais recorrentes e filtrar os que possuem uma observação única para assim realizar uma análise completa.

Dentre os feriados que [dão folga] em dia letivo segundo o site no qual foram extraídos os dados, estão listados como:

- Public Holiday
- National Holiday
- Government Holiday
- State Holiday	
- State Legal Holiday	

Para realizar uma separação desses tipos de feriado dos demais (como apenas sazonais ou comemorativos nos quais os negócios funcionam normalmente), vamos agrupá-los em uma nova coluna no dataset.

In [29]:
# lista de feriados que realmente ''''Realmente tiram dia de folga'''''

lista_nfac = ['Public Holiday',
        'National Holiday',
        'Government Holiday',
        'State Holiday',
        'State Legal Holiday']

# lista auxiliar para indicar feriados
aux = []

# buscando 'matches' entre a lista de feriados e a coluna que possue todos os tipos de feirado
for s in df_feriados['tipo']:
        if any(xs in s for xs in lista_nfac):
                aux.append(1)
        else:
                aux.append(0)

# adicionando a coluna indicadora de feriados no dataframe
df_feriados['indicadora'] = aux


Vamos agora agrupar os dados por país e indicadora de feriado a primeiro nível

In [38]:
df_feriados.describe()

,indicadora
count,7318.000000
mean,0.313200
std,0.463827
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000
